<a href="https://colab.research.google.com/github/gracemillard/Airbnb-Kaggle-Competition/blob/master/cin_7_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Summary of Functions **


---

**table_info**(endpoint,date,username,password),

 a preview of endpoint

---

**get_historical_data**(username,passwor,endpoint,date),
##### everything from given date range to current - dosnt need to be in cloud func 

---

**fix_singular_digit**(digit), 

adds leading 0s to a singluar number

---

**identify_date()**, requires fix_singular_date()

 identifies the start of the date range that daily func will pull

---

**get_daily_data**(username,password,endpoint), ...also takes identify_date(),  

should be similar to get_historical but it should excecute at the first hour of the day pulling complete records for the previous day, this one goes in google cloud func

---

**reindex_name_describe_locate_csv**(df, endpoint),...also takes identify_date(),

cleans up df, collects info to QA the data for this day, assigns a name and location for csv 


---

**get_all_historical_data**(endpoint_list, username, password, startdate) also takes historical_reindex function and get_historical_data()

## imports and credentials

### imports

In [2]:
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
import json
from google.cloud import storage
import io
import time as tme
from datetime import datetime, time, timedelta
import pytz 
import re
import time as tme
from google.cloud import storage
from google.colab import drive


### importing credentials thru shared gdrive mounted to colab

In [31]:
from google.colab import drive
drive.mount('/content/gdrive')
sys.path.append('/content/gdrive/Shareddrives/Latticework/keys')
!cp gdrive/Shareddrives/Latticework/keys/fellow_creds.py .
import fellow_creds

Mounted at /content/gdrive


In [40]:
username=fellow_creds.username
password=fellow_creds.password

'FellowIndustrie2US'

In [28]:
#drive.flush_and_unmount()  #do this to unmount gdrive

In [ ]:
#username=
#password=

dr='2021-04-01T00:00:00Z'  #april 1 is all the historical data we'll need
endpoint_list=['Contacts','Branches','ProductCategories','ProductOptions','SalesOrders','Payments','Quotes','PurchaseOrders','Stock','BranchTransfers','Adjustments','CreditNotes','BomMasters','ProductionJobs','Users']
ep='Payments'

## Table info

In [ ]:
def table_info(endpoint, daterange,username,password):
  #response=requests.get("https://api.cin7.com/api/v1/{}?where=createdDate>='{}'&order=createdDate ASC&page={}".format(endpoint,daterange,1)
  #, auth=HTTPBasicAuth(username, password))

  response=requests.get("https://api.cin7.com/api/v1/{}?where=modifiedDate>='{}'&order=modifiedDate ASC&page={}".format(endpoint,daterange,1)
  , auth=HTTPBasicAuth(username, password))

  k=response.json()[0]

  ak=k.keys()
  all_keys=list(ak)

  lk=len(response.json())
   
  whole_response=response.json()
  n=pd.DataFrame(whole_response)
  tmin=n.createdDate.min()
  tmax= n.createdDate.max()

  response.json
  #n['cdate']=pd.to_datetime(n['createdDate'])
  n['cdate']=pd.to_datetime(n['modifiedDate'])
  t_day_dist=n.cdate.apply(lambda dt: dt.date()).groupby([n.cdate.apply(lambda dt: dt.day)]).nunique()


  print('the keys are {}'.format(all_keys))
  print('the table length is {}'.format(lk))
  print('the table spans {} thru {}'.format(tmin,tmax))
  print('the distribution of days below')
  print(t_day_dist)

  return t_day_dist

## Historical data

In [ ]:
def get_historical_data(uname,password,endpoint,daterange):

   page_count=1
   current_len=50
   res_df = pd.DataFrame()

   while current_len>=50: #if first iteration or the length of previous one is 50 (50 = max)  

     next_response=requests.get("https://api.cin7.com/api/v1/{}?where=modifiedDate>='{}'&order=modifiedDate ASC&page={}".format(endpoint,daterange,page_count), 
                         auth=HTTPBasicAuth(username, password)) #then make another response for next page
                                                                 

     page_count +=1
   
        
     print(page_count-1)                                     #print page number of request that was just made
     w_res=next_response.json()        
     print(w_res)                                            #print its response
     jw_res=pd.DataFrame(w_res) 
     print(jw_res)                                                    #wonder what happens if u try to make an empty response (empty list) into a pd.df>> ...it works just fine
     res_df=res_df.append(jw_res)    
     print(res_df)                                               #empty df also appends to an exisitng df ok
     current_len=len(jw_res)      
     print(current_len)                                                  #empty response will also show 0
     tme.sleep(1.2)

                                                                        #i still cant find issues with this function and how it handles empty responses
     #res_df.to_cvs()
   return res_df
   

## Singular digits and identify date

In [ ]:
def fix_singular_digit(digit):
   if digit<10:
     digit='0{}'.format(digit)
   return digit
  



def identify_date():

  #time zone info
  tz = pytz.timezone("America/Los_Angeles")
  yesterday = datetime.now(tz).date() - timedelta(days=1, hours=0)
  midnight = tz.localize(datetime.combine(yesterday, time(0, 0)), is_dst=None) #la yesterday at midnight

  
  #now I need to fit it into this date input format that the api takes '2021-04-01T00:00:00Z'

  #save them as strings
  y_month=midnight.month
  y_day=midnight.day
  y_year=midnight.year
   
  #add leading zeros to the month and day
  fixed_month=fix_singular_digit(y_month)
  fixed_day=fix_singular_digit(y_day)

  yesterday_at_midnight='{}-{}-{}T00:00:00Z'.format(y_year,fixed_month,fixed_day)


  return yesterday_at_midnight

## Daily Data

In [ ]:
def get_daily_data(uname,password,endpoint):
   page_count=1
   current_len=50
   res_df = pd.DataFrame()
   yesterday_at_midnight=identify_date()


  #now plugging yesterday_at_midnight into the request

   while current_len>=50:
     #next_response=requests.get("https://api.cin7.com/api/v1/{}?where=createdDate>='{}'&order=createdDate ASC&page={}".format(endpoint,yesterday_at_midnight,page_count), 
     #                    auth=HTTPBasicAuth(username, password))
     
     next_response=requests.get("https://api.cin7.com/api/v1/{}?where=modifiedDate>='{}'&order=modifiedDate ASC&page={}".format(endpoint,yesterday_at_midnight,page_count), 
                         auth=HTTPBasicAuth(username, password))
     page_count +=1
        
     print(page_count-1)
     w_res=next_response.json()
     print(w_res)
     jw_res=pd.DataFrame(w_res)
     res_df=res_df.append(jw_res)
     current_len=len(jw_res)
     tme.sleep(1.2)

   
     #res_df.to_cvs()

     
   return res_df
   

## Reindexing, naming, describing, locating, 

In [ ]:
def reindex_name_describe_locate_csv_daily(big_df,endpoint):
  
  big_df.reset_index(drop=True, inplace=True)
  yesterday_at_midnight=identify_date()


  year=re.findall(r'^\d{4}',yesterday_at_midnight)
  month=re.findall(r'(?<=[-])\d{2}(?=[-])',yesterday_at_midnight)
  day=re.findall(r'(?<=[-])\d{2}(?=[T])',yesterday_at_midnight)
  hour_min_second=re.findall(r'(?<=[T])\S{8}(?=[Z])',yesterday_at_midnight)

  #mindate=min(big_df['createdDate'])
  #maxdate=max(big_df['createdDate'])

  mindate=min(big_df['createdDate'])
  maxdate=max(big_df['createdDate'])


  print("df for {}/{}/{} at {} is {} in length".format(month[0],day[0],year[0],hour_min_second[0],len(big_df)))
  print('its dates span {} to {}'.format(mindate,maxdate))
  

  file_name="{}_{}_{}_{}".format(month[0],day[0],year[0],endpoint)
  
  print('file name will be {}.csv'.format(file_name))

  #big_df.to_csv('folder location in bucket???///naming convention.csv{}'.format(date_name_string))

  return big_df

In [ ]:
def reindex_name_describe_locate_csv_historical(big_df,endpoint):

  if len(big_df)==0:
    print('yes the {} endpoint response is null !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1'.format(endpoint))
  else:
    print('not null!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')


  
    big_df.reset_index(drop=True, inplace=True)

    print('input df looks like')
    print(big_df)

    print('does input df have a modified date feild')

    print(big_df['modifiedDate'])
    
    
    mindate=min(big_df['modifiedDate'])
    maxdate=max(big_df['modifiedDate'])

    min_month=month=re.findall(r'(?<=[-])\d{2}(?=[-])',mindate)
    min_day=re.findall(r'(?<=[-])\d{2}(?=[T])',mindate)
    min_year=re.findall(r'^\d{4}',mindate)

    max_month=month=re.findall(r'(?<=[-])\d{2}(?=[-])',maxdate)
    max_day=re.findall(r'(?<=[-])\d{2}(?=[T])',maxdate)
    max_year=re.findall(r'^\d{4}',maxdate)


    print("df for {}/{}/{} thru {}/{}/{} for {} endpoint is {} in length".format(min_month[0],min_day[0],min_year[0],max_month[0],max_day[0],max_year[0],endpoint, len(big_df)))
    

    file_name="{}_{}_backlog_{}".format(min_month[0],min_year[0],endpoint)
    
    print('file name will be {}.csv'.format(file_name))

    #big_df.to_csv('folder location in bucket???///naming convention.csv{}'.format(date_name_string))

  return big_df

## Get All Historical Data

In [ ]:
def get_all_historical_data(endpoint_list, username, password, startdate):
  for i in range(len(endpoint_list)):
    ep=endpoint_list[i]
    print(ep)
    ep_specific_hist_df=get_historical_data(username,password,ep,startdate)
    print('first function worked!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print("second function input df is :")
    print(ep_specific_hist_df)
    processed_ep_specific_hist_df=reindex_name_describe_locate_csv_historical(ep_specific_hist_df,ep)
    print('made it thru second function')
    print(processed_ep_specific_hist_df)


## **Ongoing Concern**

Im still concerned about the date cause technically we are working with whatever timezone the api is working with, me grabbing the la timezone info is doing absolutly nothing if im just reformating it and giving it back to the api in its desired form

altho the time dosnt matter much as long as it aligns with the time the function is scheduled to run....

alas i will have to go back into the doccumentation and read to figure out how the api is measuring time (gmt or utc....+-timezone???)

## **Current Focus**

In this notebook i will list out all the endpoint names and throw historical april data into bucket

Then i will put daily function in cloud func

I wanted to write csv's directly into bucket but [this](https://stackoverflow.com/questions/56596951/save-pandas-data-frame-to-google-cloud-bucket/56787083) will have to do: making a temp csv , I guess i should have a folder for each endpoint

# Issues and Noteable things

### Endpoints without date

ProductCatergories   - id is kinda sequential, is also very small (length 33)

Users      - also dosnt have a create or modified date, looks like Fellow employees (all 30 entries have fellow email addresses)

both endpoints can still be grabbed with the get_historical_data function

### Very large important endpoints

sales orders and payments are largest, stock is also up there

## Issues

bommasters there is nothing after april, 1 it has been unused since 2021-03-22T21:13:24Z
my historical function fails at bommasters even tho datemodified exisits because there is no April data .
I need to fix this cause what if it fails for my daily functions when nothing has been added that day,
 i need to test and adjust the reindex_name_describe_locate_csv functions i think theres an issue with the logic there 
 I should add something like: if the df is empty dont do anything (dont name it or generate csv)

 same but kinda diffrent issue with production jobs and users

 what if these endpoints have exactly 50 that would fuck up the logic with the 
 first function

 ---



 I think the first function was fine, I resolved it by adding a conditional to the second fumction
 ready to add a line to create new bucket folders (if not already exist) and stow the retroactive data in proper folders

 ---

 then can implement similar thing with the fresh (daily) data, fo which functions will live in the cloud function

In [ ]:
def get_all_historical_data(endpoint_list, username, password, startdate):
  for i in range(len(endpoint_list)):
    ep=endpoint_list[i]
    print(ep)
    ep_specific_hist_df=get_historical_data(username,password,ep,startdate)
    processed_ep_specific_hist_df=reindex_name_describe_locate_csv_historical(ep_specific_hist_df,ep)



  


In [ ]:
endpoint_list_1=['Contacts','Branches','ProductCategories','ProductOptions','SalesOrders','Payments','Quotes','PurchaseOrders','Stock','BranchTransfers','Adjustments','CreditNotes','BomMasters','ProductionJobs','Users']
endpoint_list_2=['Contacts','Branches','ProductOptions','SalesOrders','Payments','Quotes','PurchaseOrders','Stock','BranchTransfers','Adjustments','CreditNotes','BomMasters','ProductionJobs','Users']
endpoint_list_3=['Quotes','PurchaseOrders','Stock','BranchTransfers','Adjustments','CreditNotes','BomMasters','ProductionJobs','Users']
endpoint_list_4=['BranchTransfers','Adjustments','CreditNotes','BomMasters','ProductionJobs','Users']
endpoint_list_5=['BomMasters','ProductionJobs','Users']
endpoint_list_6=['ProductionJobs','Users']
endpoint_list_7=['Users']

In [ ]:
test=requests.get("https://api.cin7.com/api/v1/{}".format('Users'), 
                         auth=HTTPBasicAuth(username, password))

In [ ]:
test=get_historical_data(username,password,'Users',dr)

1
[{'id': 76537, 'isActive': True, 'firstName': 'Erica ', 'lastName': 'Chenard', 'jobTitle': None, 'email': 'erica@fellowproducts.com'}, {'id': 76366, 'isActive': True, 'firstName': 'Catherine', 'lastName': 'Li', 'jobTitle': 'Accounting Specialist', 'email': 'cathy@fellowproducts.com'}, {'id': 74295, 'isActive': True, 'firstName': 'Andrea', 'lastName': 'Gonzalez', 'jobTitle': 'Director of Ecomm', 'email': 'andrea@fellowproducts.com'}, {'id': 74294, 'isActive': True, 'firstName': 'Brian', 'lastName': 'Quan', 'jobTitle': 'Ecomm Coordinator', 'email': 'brian@fellowproducts.com'}, {'id': 74220, 'isActive': True, 'firstName': 'Caroline', 'lastName': 'de Figueiredo', 'jobTitle': 'International Account Manager', 'email': 'caroline@fellowproducts.com'}, {'id': 74221, 'isActive': True, 'firstName': 'Amanda', 'lastName': 'Schuman', 'jobTitle': 'International Sales Coordinator', 'email': 'amanda@fellowproducts.com'}, {'id': 74219, 'isActive': True, 'firstName': 'Kat', 'lastName': 'Miranda', 'jobT

In [ ]:
test

,id,isActive,firstName,lastName,jobTitle,email
0,76537,True,Erica,Chenard,None,erica@fellowproducts.com
1,76366,True,Catherine,Li,Accounting Specialist,cathy@fellowproducts.com
2,74295,True,Andrea,Gonzalez,Director of Ecomm,andrea@fellowproducts.com
3,74294,True,Brian,Quan,Ecomm Coordinator,brian@fellowproducts.com
4,74220,True,Caroline,de Figueiredo,International Account Manager,caroline@fellowproducts.com
5,74221,True,Amanda,Schuman,International Sales Coordinator,amanda@fellowproducts.com
6,74219,True,Kat,Miranda,"Sales Coordinator, Corporate and Promotions",kat@fellowproducts.com
7,72989,True,Alex,Saccone,None,alex@fellowproducts.com
8,71988,True,Sophie,Frank,Analytics Manager,sophie@fellowproducts.com
9,71919,True,Rae,Smith-Joseph,Logistics Coordinator,rae@fellowproducts.com


In [ ]:
test.json()

[{'email': 'erica@fellowproducts.com',
  'firstName': 'Erica ',
  'id': 76537,
  'isActive': True,
  'jobTitle': None,
  'lastName': 'Chenard'},
 {'email': 'cathy@fellowproducts.com',
  'firstName': 'Catherine',
  'id': 76366,
  'isActive': True,
  'jobTitle': 'Accounting Specialist',
  'lastName': 'Li'},
 {'email': 'andrea@fellowproducts.com',
  'firstName': 'Andrea',
  'id': 74295,
  'isActive': True,
  'jobTitle': 'Director of Ecomm',
  'lastName': 'Gonzalez'},
 {'email': 'brian@fellowproducts.com',
  'firstName': 'Brian',
  'id': 74294,
  'isActive': True,
  'jobTitle': 'Ecomm Coordinator',
  'lastName': 'Quan'},
 {'email': 'caroline@fellowproducts.com',
  'firstName': 'Caroline',
  'id': 74220,
  'isActive': True,
  'jobTitle': 'International Account Manager',
  'lastName': 'de Figueiredo'},
 {'email': 'amanda@fellowproducts.com',
  'firstName': 'Amanda',
  'id': 74221,
  'isActive': True,
  'jobTitle': 'International Sales Coordinator',
  'lastName': 'Schuman'},
 {'email': 'kat@f

In [ ]:
get_all_historical_data(endpoint_list_7,username,password,dr)

Users
1
[{'id': 76537, 'isActive': True, 'firstName': 'Erica ', 'lastName': 'Chenard', 'jobTitle': None, 'email': 'erica@fellowproducts.com'}, {'id': 76366, 'isActive': True, 'firstName': 'Catherine', 'lastName': 'Li', 'jobTitle': 'Accounting Specialist', 'email': 'cathy@fellowproducts.com'}, {'id': 74295, 'isActive': True, 'firstName': 'Andrea', 'lastName': 'Gonzalez', 'jobTitle': 'Director of Ecomm', 'email': 'andrea@fellowproducts.com'}, {'id': 74294, 'isActive': True, 'firstName': 'Brian', 'lastName': 'Quan', 'jobTitle': 'Ecomm Coordinator', 'email': 'brian@fellowproducts.com'}, {'id': 74220, 'isActive': True, 'firstName': 'Caroline', 'lastName': 'de Figueiredo', 'jobTitle': 'International Account Manager', 'email': 'caroline@fellowproducts.com'}, {'id': 74221, 'isActive': True, 'firstName': 'Amanda', 'lastName': 'Schuman', 'jobTitle': 'International Sales Coordinator', 'email': 'amanda@fellowproducts.com'}, {'id': 74219, 'isActive': True, 'firstName': 'Kat', 'lastName': 'Miranda',

KeyError: ignored

In [ ]:
storage_client = storage.Client()

# Creates the new bucket
bucket_name='test_bucket'
bucket = storage_client.create_bucket(bucket_name)

print("Bucket {} created.".format(bucket.name))